## 3. Assignment
### 다음 미완성 코드를 활용해 새로운 data를 학습해보자
* Training data image 
  + Shape: (?, 3, 64, 128)
      - 여러장의, RGB, 64x128 size의 이미지라고 가정하자
* Test data image
  + Shape: (?, 3, ?, ?)
      - 여러장의, RGB, size를 알 수 없는 이미지라고 가정하자
* Labels
  + image의 class는 2가지 이다
  + data folder의 구성을 참고하자
* Data folder 위치
  + 노트북 github의 data folder 2개를 적당한 위치에 카피해 사용한다




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# Model
##### 지난 assignment에서 작성한 모델을 활용한다 
##### 바뀐 image input에 맞게 변형한다
##### 바뀐 image의 size는 64x128이며, 흑백이 아닌 RGB 3개의 채널임에 유의한다
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 Input shape=(?, 1, 28, 28)
        #    Conv     -> (?, 32, 28, 28)
        #    Pool     -> (?, 32, 14, 14)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # L2 Input shape=(?, 32, 14, 14)
        #    Conv      ->(?, 64, 14, 14)
        #    Pool      ->(?, 64, 7, 7)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        # L3 Input shape=(?, 64, 7, 7)
        #    Conv      ->(?, 128, 7, 7)
        #    Pool      ->(?, 128, 4, 4)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1)) 
        
        # L4 Linear 128x4x4 inputs -> 625 outputs
        self.Linear = torch.nn.Linear(19584, 625, bias=True)  # 128로 바꿔보기
        torch.nn.init.xavier_uniform_(self.Linear.weight)
        self.layer4 = torch.nn.Sequential(
            self.Linear,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.5))
       
        # L5 Linear 625 inputs -> 10 outputs
        self.layer5 = torch.nn.Linear(625, 2, bias=True)
        torch.nn.init.xavier_uniform_(self.layer5.weight)
        # softmax는 Cross Entropy Loss에 포함되어있음.
        

    def forward(self, x):
        # print("adsfasdfasfsdf")
        # print(x.shape)           # torch.Size([8, 3, 64, 128])
        out = self.layer1(x)     # [8, 128, 16, 32]
        # print(out.shape)
        out = self.layer2(out)   
        # print(out.shape)
        out = self.layer3(out)
        # print(out.shape)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        # print(out.shape)
        out = self.layer4(out)
        # print(out.shape)
        out = self.layer5(out)
        # print(out.shape)
        out = out.view(out.size(0), -1) 
        # print(out.shape)
        return out

# parameters
##### 적당히 수정한다
learning_rate = 0.001
training_epochs = 15

# contruct model
model = CNN().to(device)

# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# train
##### 직접 dataset을 만들어 학습할 때 자주 사용되는 함수이니 기능을 숙지한다
##### pytorch 공식 홈페이지에서 관련 함수의 정보를 직접 찾아보는 것을 추천한다
trans = transforms.Compose([transforms.ToTensor()])
train_data = dsets.ImageFolder(root='/content/drive/MyDrive/[21학년도 2학기]인공지능응용/train_data', transform=trans)
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True)


# for X, Y in data_loader:
#   print(X.shape)    # torch.Size([8, 3, 64, 128])
#   print(Y.shape)    # torch.Size([8])
#   break


cuda


In [3]:

total_batch = len(data_loader)

##### 학습이 이루어지는 loop
##### 지난 source code를 그대로 사용해도 되지만, 연습삼아 직접 작성해보자
for epoch in range(training_epochs):
    for X, Y in data_loader:
        X = X .to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)

        # print(Y)
        # print(hypothesis)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
      
    
    cost = torch.mean(cost)
    print('Epoch {:4d}/{} Cost: {:.9f}'.format(epoch+1, training_epochs, cost.item()))


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch    1/15 Cost: 0.000000000
Epoch    2/15 Cost: 0.000000000
Epoch    3/15 Cost: 0.000000000
Epoch    4/15 Cost: 0.044655304
Epoch    5/15 Cost: 0.000000119
Epoch    6/15 Cost: 0.000000000
Epoch    7/15 Cost: 0.000000000
Epoch    8/15 Cost: 0.000000000
Epoch    9/15 Cost: 0.000000000
Epoch   10/15 Cost: 0.000000000
Epoch   11/15 Cost: 0.000000000
Epoch   12/15 Cost: 0.000000000
Epoch   13/15 Cost: 0.000000000
Epoch   14/15 Cost: 0.000000000
Epoch   15/15 Cost: 0.000000000


In [4]:
print(Y)
print(hypothesis)

tensor([1, 0, 1, 0, 0, 0, 0], device='cuda:0')
tensor([[-83.5229,  50.8721],
        [  6.8854,  -6.0845],
        [-88.2169,  40.6859],
        [ 11.4427,  -9.8422],
        [ 12.2441, -12.7569],
        [  7.6094,  -6.8586],
        [  9.6085,  -7.2175]], device='cuda:0', grad_fn=<ViewBackward>)


In [7]:

# test
##### 직접 dataset을 만들어 학습할 때 자주 사용되는 함수이니 기능을 숙지한다
##### pytorch 공식 홈페이지에서 관련 함수의 정보를 직접 찾아보는 것을 추천한다
trans=transforms.Compose([transforms.Resize((64,128)),transforms.ToTensor()])
test_data = dsets.ImageFolder(root='/content/drive/MyDrive/[21학년도 2학기]인공지능응용/test_data', transform=trans)
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

with torch.no_grad():
##### 결과 test가 이루어지는 loop
##### 지난 source code를 그대로 사용해도 되지만, 연습삼아 직접 작성해보자
  for X, Y in data_loader:
    X_test = X.to(device)
    Y_test = Y.to(device)

    prediction = model(X_test)
    correct = torch.argmax(prediction, 1) == Y_test
    accuracy = correct.float().mean()
  print('Accuracy:', accuracy.item())



Accuracy: 1.0
